In [35]:
import arcpy
import os
from dotenv import load_dotenv
from pathlib import Path
import json
import arcpy
from arcpy.da import InsertCursor
from shapely.geometry import Point

###################################################
########## path to geodatabase ####################
load_dotenv() # --> load in path via env variable 
raw_path:str = os.getenv('GDB_PATH')
GDB_PATH:str = r"{}".format(raw_path)
if arcpy.Exists(GDB_PATH): 
    print(f"Geodatabase exists: {GDB_PATH}")
else:
    print(f"Geodatabase DNE: {GDB_PATH}")

###################################################
#####  Spatial Refernce for Project: WGS 84  ######

SPATIAL_REF= arcpy.SpatialReference(4326) # WGS 84 (latitude/longitude)
#SPATIAL_REF = arcpy.SpatialReference(26918)

###################################################
#######     Paths to all json files         #######
"""
All Json files contain points representing:
- grocery stores
- convenience stores
- cornerstores
- cannabis and liquor stores
- farmers markets
- fast food restaurants
- food banks
- mini marts
"""
GROCERY_JSON_PATH:object = Path(r"..\data_collection\grocery_edited.json")
CONVENIENCE_JSON_PATH:object= Path(r"..\data_collection\convenience.json")
CORNERSTORE_JSON_PATH:object = Path(r"..\data_collection\cornerstore.json")
DRUGS_NLIQUOR_JSON_PATH:object = Path(r"..\data_collection\drugs_nliquor.json")
FARM_MARKER_JSON_PATH:object = Path(r"..\data_collection\farm_market.json")
FAST_FOOD_JSON_PATH:object = Path(r"..\data_collection\fast_food.json")
FOOD_BANK_JSON_PATH:object = Path(r"..\data_collection\food_bank.json")
MINI_MART_JSON_PATH:object = Path(r"..\data_collection\mini_mart.json")

PATH_LIST:list[object] = [GROCERY_JSON_PATH,
             CONVENIENCE_JSON_PATH,
             CORNERSTORE_JSON_PATH,
             DRUGS_NLIQUOR_JSON_PATH,
             FARM_MARKER_JSON_PATH,
             FAST_FOOD_JSON_PATH,
             FOOD_BANK_JSON_PATH,
             MINI_MART_JSON_PATH
             ]

for path in PATH_LIST:
    if path.exists():
        print(f"{path.name} exists")
    else:
        print(f"{path.name} doesn't exist")

BUFFER_DISTANCE = "0.5 miles"


Geodatabase exists: C:\Users\jbfre\Documents\ArcGIS\Projects\From_Desert_2_Oasis\From_Desert_2_Oasis.gdb
grocery_edited.json exists
convenience.json exists
cornerstore.json exists
drugs_nliquor.json exists
farm_market.json exists
fast_food.json exists
food_bank.json exists
mini_mart.json exists


### Grocery Store Data Processing

In [36]:
"""
Grocery Store Data --> Creating a feature layer that has all the coordinates of grocery stores.

Each point will also have a 0.5 mile radius (a colored circle visually) that will signify the "reach" that a grocery store has
as a proof of concept. 
"""

### Create the grocery store feature class ###
GROCERY_FC = "grocery_store_fc"
GROCERY_FC_PATH = os.path.join(GDB_PATH,GROCERY_FC)

### Make sure to define the coordinate system / spatial reference of any feature class that is created
arcpy.DefineProjection_management(GROCERY_FC_PATH, SPATIAL_REF)


### Verify that the grocery feature class was successfully created ###
if arcpy.Exists(GROCERY_FC_PATH):
    arcpy.Delete_management(GROCERY_FC_PATH)
    print("Grocery Feature Class already exists... deleting and creating a new one ")
    arcpy.CreateFeatureclass_management(GDB_PATH,GROCERY_FC,"POINT")
    
else:
    arcpy.CreateFeatureclass_management(GDB_PATH,GROCERY_FC,"POINT")
    if arcpy.Exists(GROCERY_FC_PATH):
        print("Grocery Feature Class created successfully")
    else:
        print("Grocer Feature Class DNE: Error")


Grocery Feature Class already exists... deleting and creating a new one 


In [37]:
### Check existing fields in Grocery Feature Class ###
print("Fields in the feature class:")
for field in arcpy.ListFields(GROCERY_FC_PATH):
    print(field.name)

### Add fields for storing coordinates for easier calculations later ###
arcpy.AddField_management(GROCERY_FC_PATH, "X_Coord", "DOUBLE")
arcpy.AddField_management(GROCERY_FC_PATH, "Y_Coord", "DOUBLE")
arcpy.AddField_management(GROCERY_FC_PATH,"Address","TEXT")

required_fields:list[str] = ["Address", "X_Coord", "Y_Coord"]
for required_field in required_fields:
    if any(field.name == required_field for field in arcpy.ListFields(GROCERY_FC_PATH)):
        print(f"The field '{required_field}' exists.")
    else:
        print(f"The field '{required_field}' does not exist.")


Fields in the feature class:
OBJECTID
Shape
The field 'Address' exists.
The field 'X_Coord' exists.
The field 'Y_Coord' exists.


In [38]:
### Process Grocery JSON file ###
with open(GROCERY_JSON_PATH) as file:
    GROCERY_JSON = json.load(file)

# Step 3: Insert points into the feature class from the JSON data
with arcpy.da.InsertCursor(GROCERY_FC_PATH, ["SHAPE", "Address", "X_Coord", "Y_Coord"]) as cursor:
    for address, coords in GROCERY_JSON.items():
        # Create a point geometry object using the x and y coordinates
        point = arcpy.Point(coords[1], coords[0])  # coords[1] is longitude (X), coords[0] is latitude (Y)
        
        # Prepare the row with geometry and attributes
        row = [point, address, coords[1], coords[0]]  # X_Coord is longitude, Y_Coord is latitude
        
        # Insert the row into the feature class
        cursor.insertRow(row)

# Step 4: Create a feature layer from the feature class
GROCERY_FL = "grocery_fl"
if arcpy.Exists(GROCERY_FL):
    print("grocery feature layer exists... deleting and creating a new one ")
    arcpy.Delete_management(GROCERY_FL)
    arcpy.DefineProjection_management(GROCERY_FC_PATH, SPATIAL_REF)
    arcpy.MakeFeatureLayer_management(GROCERY_FC_PATH, GROCERY_FL)
else:
    print("successfully created new grocery feature layer ")
    arcpy.DefineProjection_management(GROCERY_FC_PATH, SPATIAL_REF)
    arcpy.MakeFeatureLayer_management(GROCERY_FC_PATH, GROCERY_FL)

In [39]:
### Create a grocery store buffer feature class for the 0.5 mile radius circle ###
GROCERY_BUFFER_FC = "grocery_buffer_fc"
GROCERY_BUFFER_FC_PATH = os.path.join(GDB_PATH,GROCERY_BUFFER_FC)
### Input buffer already have the points to process ###
input_fc = GROCERY_FC_PATH
GROCERY_BUFFER_LAYER = "grocery_buffer_layer"
# Step 1: Create buffer around each point (0.5 mile radius)
if arcpy.Exists(GROCERY_BUFFER_FC_PATH) or arcpy.Exists(GROCERY_BUFFER_LAYER):
    print("grocer buffer already exists... deleting and creating a new one ")
    arcpy.Delete_management(GROCERY_BUFFER_FC_PATH)
    arcpy.Delete_management(GROCERY_BUFFER_LAYER)
    arcpy.DefineProjection_management(GROCERY_FC_PATH, SPATIAL_REF)
    arcpy.Buffer_analysis(input_fc, GROCERY_BUFFER_FC_PATH, BUFFER_DISTANCE)
    # Step 2: Make the buffer result a feature layer
    
    arcpy.MakeFeatureLayer_management(GROCERY_BUFFER_FC_PATH, GROCERY_BUFFER_LAYER)

else:
    arcpy.DefineProjection_management(GROCERY_FC_PATH, SPATIAL_REF)
    arcpy.Buffer_analysis(input_fc, GROCERY_BUFFER_FC_PATH, BUFFER_DISTANCE)
    # Step 2: Make the buffer result a feature layer
    arcpy.MakeFeatureLayer_management(GROCERY_BUFFER_FC_PATH, GROCERY_BUFFER_LAYER)
    print("new grocery buffer created successfully")



# Output message to confirm the creation of the buffer layer
print(f"Feature layer '{GROCERY_BUFFER_LAYER}' created with a 0.5 mile radius around the points.")

Feature layer 'grocery_buffer_layer' created with a 0.5 mile radius around the points.


### Alternative healthy food options: Food Bank and Farmer's Market Data Processing 

In [41]:
"""
Alternative healthy food  Data --> Creating a feature layer that has all the coordinates of food banks and farmer's markets
"""

### Create the alt healthy food feature class ###
ALT_HEALTHY_FOOD_FC = "alt_healthy_food_fc"
ALT_HEALTHY_FOOD_FC_PATH = os.path.join(GDB_PATH,ALT_HEALTHY_FOOD_FC)




### Verify that the alt healthy feature class was successfully created ###
if arcpy.Exists(ALT_HEALTHY_FOOD_FC_PATH):
    arcpy.Delete_management(ALT_HEALTHY_FOOD_FC_PATH)
    print("alt healthy food feature class already exists... deleting and creating a new one ")
    arcpy.CreateFeatureclass_management(GDB_PATH,ALT_HEALTHY_FOOD_FC,"POINT")
    
else:
    arcpy.CreateFeatureclass_management(GDB_PATH,ALT_HEALTHY_FOOD_FC,"POINT")
    if arcpy.Exists(ALT_HEALTHY_FOOD_FC_PATH):
        print(" ALT_HEALTHY_FOOD_FC_PATH created successfully")
    else:
        print("ALT_HEALTHY_FOOD_FC_PATH DNE: Error")

### Make sure to define the coordinate system / spatial reference of any feature class that is created
arcpy.DefineProjection_management(ALT_HEALTHY_FOOD_FC_PATH, SPATIAL_REF)


 ALT_HEALTHY_FOOD_FC_PATH created successfully


<Result 'C:\\Users\\jbfre\\Documents\\ArcGIS\\Projects\\From_Desert_2_Oasis\\From_Desert_2_Oasis.gdb\\alt_healthy_food_fc'>

In [42]:
### Check existing fields in  Class ###
print("Fields in the feature ALT_HEALTHY_FOOD class:")
for field in arcpy.ListFields(ALT_HEALTHY_FOOD_FC_PATH):
    print(field.name)

### Add fields for storing coordinates for easier calculations later ###
arcpy.AddField_management(ALT_HEALTHY_FOOD_FC_PATH, "X_Coord", "DOUBLE")
arcpy.AddField_management(ALT_HEALTHY_FOOD_FC_PATH, "Y_Coord", "DOUBLE")
arcpy.AddField_management(ALT_HEALTHY_FOOD_FC_PATH,"Address","TEXT")

required_fields:list[str] = ["Address", "X_Coord", "Y_Coord"]
for required_field in required_fields:
    if any(field.name == required_field for field in arcpy.ListFields(ALT_HEALTHY_FOOD_FC_PATH)):
        print(f"The field '{required_field}' exists.")
    else:
        print(f"The field '{required_field}' does not exist.")


Fields in the feature ALT_HEALTHY_FOOD class:
OBJECTID
Shape
The field 'Address' exists.
The field 'X_Coord' exists.
The field 'Y_Coord' exists.


In [43]:
### Process Alt healthy food options from their JSON files ###

with open(FOOD_BANK_JSON_PATH) as file:
    FOOD_BANK_JSON = json.load(file)
with open(FARM_MARKER_JSON_PATH) as file:
    FARM_MARKER_JSON = json.load(file)

ALT_HEALTHY_FOOD_JSON = [FOOD_BANK_JSON,FARM_MARKER_JSON]

### Loop through all json files in the list ###     
for food_ops in ALT_HEALTHY_FOOD_JSON:
    # Step 3: Insert points into the feature class from the JSON data
    with arcpy.da.InsertCursor(ALT_HEALTHY_FOOD_FC_PATH, ["SHAPE", "Address", "X_Coord", "Y_Coord"]) as cursor:
        for address, coords in food_ops.items():
            # Create a point geometry object using the x and y coordinates
            point = arcpy.Point(coords[1], coords[0])  # coords[1] is longitude (X), coords[0] is latitude (Y)
            
            # Prepare the row with geometry and attributes
            row = [point, address, coords[1], coords[0]]  # X_Coord is longitude, Y_Coord is latitude
            
            # Insert the row into the feature class
            cursor.insertRow(row)

# Step 4: Create a feature layer from the feature class
ALT_HEALTHY_FOOD_FL = "alt_healthy_food_fl"
if arcpy.Exists(ALT_HEALTHY_FOOD_FL):
    arcpy.Delete_management(ALT_HEALTHY_FOOD_FL)
    arcpy.DefineProjection_management(ALT_HEALTHY_FOOD_FC_PATH, SPATIAL_REF)
    arcpy.MakeFeatureLayer_management(ALT_HEALTHY_FOOD_FC_PATH, ALT_HEALTHY_FOOD_FL)
else:
    arcpy.DefineProjection_management(ALT_HEALTHY_FOOD_FC_PATH, SPATIAL_REF)
    arcpy.MakeFeatureLayer_management(ALT_HEALTHY_FOOD_FC_PATH, ALT_HEALTHY_FOOD_FL)

### Drugs 'N Liquor Data Processing

In [53]:
"""
Cannabis and Liquor  Data --> Creating a feature layer that has all the coordinates of Cannabis shops and liquor
"""

### Create the cannabis & liquor feature class ###
DRUGS_NLIQUOR_FC = "drugs_nliquor_fc"
DRUGS_NLIQUOR_FC_PATH = os.path.join(GDB_PATH,DRUGS_NLIQUOR_FC)




### Verify that the drugs n liquor class was successfully created ###
if arcpy.Exists(DRUGS_NLIQUOR_FC_PATH):
    arcpy.Delete_management(DRUGS_NLIQUOR_FC_PATH)
    print("drugs and liquor feature class already exists... deleting and creating a new one ")
    arcpy.CreateFeatureclass_management(GDB_PATH,DRUGS_NLIQUOR_FC,"POINT")
    
else:
    arcpy.CreateFeatureclass_management(GDB_PATH,DRUGS_NLIQUOR_FC,"POINT")
    if arcpy.Exists(DRUGS_NLIQUOR_FC_PATH):
        print(" DRUGS_NLIQUOR_FC_PATH created successfully")
    else:
        print("DRUGS_NLIQUOR_FC_PATH DNE: Error")

### Make sure to define the coordinate system / spatial reference of any feature class that is created
arcpy.DefineProjection_management(DRUGS_NLIQUOR_FC_PATH, SPATIAL_REF)

drugs and liquor feature class already exists... deleting and creating a new one 


<Result 'C:\\Users\\jbfre\\Documents\\ArcGIS\\Projects\\From_Desert_2_Oasis\\From_Desert_2_Oasis.gdb\\drugs_nliquor_fc'>

In [56]:
### Check existing fields in  drugs n liquor feature Class ###
print("Fields in the feature drugs n liquor feature class:")
for field in arcpy.ListFields(DRUGS_NLIQUOR_FC_PATH):
    print(field.name)

### Add fields for storing coordinates for easier calculations later ###
arcpy.AddField_management(DRUGS_NLIQUOR_FC_PATH, "X_Coord", "DOUBLE")
arcpy.AddField_management(DRUGS_NLIQUOR_FC_PATH, "Y_Coord", "DOUBLE")
arcpy.AddField_management(DRUGS_NLIQUOR_FC_PATH,"Address","TEXT")

required_fields:list[str] = ["Address", "X_Coord", "Y_Coord"]
for required_field in required_fields:
    if any(field.name == required_field for field in arcpy.ListFields(DRUGS_NLIQUOR_FC_PATH)):
        print(f"The field '{required_field}' exists.")
    else:
        print(f"The field '{required_field}' does not exist.")

Fields in the feature drugs n liquor feature class:
OBJECTID
Shape
X_Coord
Y_Coord
Address
The field 'Address' exists.
The field 'X_Coord' exists.
The field 'Y_Coord' exists.


In [57]:
### Process drugs n liquor JSON file ###
with open(DRUGS_NLIQUOR_JSON_PATH) as file:
    DRUGS_NLIQUOR_JSON = json.load(file)

# Step 3: Insert points into the feature class from the JSON data
with arcpy.da.InsertCursor(DRUGS_NLIQUOR_FC_PATH, ["SHAPE", "Address", "X_Coord", "Y_Coord"]) as cursor:
    for address, coords in DRUGS_NLIQUOR_JSON.items():
        # Create a point geometry object using the x and y coordinates
        point = arcpy.Point(coords[1], coords[0])  # coords[1] is longitude (X), coords[0] is latitude (Y)
        
        # Prepare the row with geometry and attributes
        row = [point, address, coords[1], coords[0]]  # X_Coord is longitude, Y_Coord is latitude
        
        # Insert the row into the feature class
        cursor.insertRow(row)

# Step 4: Create a feature layer from the feature class
DRUGS_NLIQUOR_FL = "drugs_nliquor_fl"
if arcpy.Exists(DRUGS_NLIQUOR_FL):
    print("drugs and liquor feature layer exists... deleting and creating a new one ")
    arcpy.Delete_management(DRUGS_NLIQUOR_FL)
    arcpy.DefineProjection_management(DRUGS_NLIQUOR_FC_PATH, SPATIAL_REF)
    arcpy.MakeFeatureLayer_management(DRUGS_NLIQUOR_FC_PATH, DRUGS_NLIQUOR_FL)
else:
    print("successfully created drugs and liquor feature layer ")
    arcpy.DefineProjection_management(DRUGS_NLIQUOR_FC_PATH, SPATIAL_REF)
    arcpy.MakeFeatureLayer_management(DRUGS_NLIQUOR_FC_PATH, DRUGS_NLIQUOR_FL)

drugs and liquor feature layer exists... deleting and creating a new one 


In [58]:
### Create a drugs n liquor buffer feature class for the 0.5 mile radius circle ###
DRUGS_NLIQUOR_BUFFER_FC = "drugs_nliquor_buffer_fc"
DRUGS_NLIQUOR_BUFFER_FC_PATH = os.path.join(GDB_PATH,DRUGS_NLIQUOR_BUFFER_FC)
### Input buffer already have the points to process ###
input_fc = DRUGS_NLIQUOR_FC_PATH
DRUGS_NLIQUOR_BUFFER_LAYER = "drugs_nliquor_layer"
# Step 1: Create buffer around each point (0.5 mile radius)
if arcpy.Exists(DRUGS_NLIQUOR_FC_PATH) or arcpy.Exists(DRUGS_NLIQUOR_BUFFER_LAYER):
    print("drugs n liquor buffer already exists... deleting and creating a new one ")
    arcpy.Delete_management(DRUGS_NLIQUOR_BUFFER_FC_PATH)
    arcpy.Delete_management(DRUGS_NLIQUOR_BUFFER_LAYER)
    arcpy.DefineProjection_management(DRUGS_NLIQUOR_FC_PATH, SPATIAL_REF)
    arcpy.Buffer_analysis(input_fc, DRUGS_NLIQUOR_BUFFER_FC_PATH, BUFFER_DISTANCE)
    # Step 2: Make the buffer result a feature layer
    
    arcpy.MakeFeatureLayer_management(DRUGS_NLIQUOR_BUFFER_FC_PATH, DRUGS_NLIQUOR_BUFFER_LAYER)

else:
    arcpy.DefineProjection_management(DRUGS_NLIQUOR_FC_PATH, SPATIAL_REF)
    arcpy.Buffer_analysis(input_fc, DRUGS_NLIQUOR_BUFFER_FC_PATH, BUFFER_DISTANCE)
    # Step 2: Make the buffer result a feature layer
    arcpy.MakeFeatureLayer_management(DRUGS_NLIQUOR_BUFFER_FC_PATH, DRUGS_NLIQUOR_BUFFER_LAYER)
    print("new drugs n liquor buffer created successfully")



# Output message to confirm the creation of the buffer layer
print(f"Feature layer '{DRUGS_NLIQUOR_BUFFER_LAYER}' created with a 0.5 mile radius around the points.")

drugs n liquor buffer already exists... deleting and creating a new one 
Feature layer 'drugs_nliquor_layer' created with a 0.5 mile radius around the points.


### Convenience, Corner, and Mini-Mart Data Processing

In [59]:
"""
Unhealthy Food Option Data --> Creating a feature layer that has all the coordinates of food convenience stores, 
corner stores, and mini-marts.
"""

### Create the unhealthy food feature class ###
UNHEALTHY_FOOD_FC = "unhealthy_food_fc"
UNHEALTHY_FOOD_FC_PATH = os.path.join(GDB_PATH,UNHEALTHY_FOOD_FC)




### Verify that the unhealthy feature class was successfully created ###
if arcpy.Exists(UNHEALTHY_FOOD_FC_PATH):
    arcpy.Delete_management(UNHEALTHY_FOOD_FC_PATH)
    print("unhealthy food feature class already exists... deleting and creating a new one ")
    arcpy.CreateFeatureclass_management(GDB_PATH,UNHEALTHY_FOOD_FC,"POINT")
    
else:
    arcpy.CreateFeatureclass_management(GDB_PATH,UNHEALTHY_FOOD_FC,"POINT")
    if arcpy.Exists(UNHEALTHY_FOOD_FC_PATH):
        print(" UNHEALTHY_FOOD_FC_PATH created successfully")
    else:
        print("UNHEALTHY_FOOD_FC_PATH DNE: Error")

### Make sure to define the coordinate system / spatial reference of any feature class that is created
arcpy.DefineProjection_management(UNHEALTHY_FOOD_FC_PATH, SPATIAL_REF)

 UNHEALTHY_FOOD_FC_PATH created successfully


<Result 'C:\\Users\\jbfre\\Documents\\ArcGIS\\Projects\\From_Desert_2_Oasis\\From_Desert_2_Oasis.gdb\\unhealthy_food_fc'>

In [60]:
### Check existing fields in  Class ###
print("Fields in the feature UNHEALTHY_FOOD feature class:")
for field in arcpy.ListFields(UNHEALTHY_FOOD_FC_PATH):
    print(field.name)

### Add fields for storing coordinates for easier calculations later ###
arcpy.AddField_management(UNHEALTHY_FOOD_FC_PATH, "X_Coord", "DOUBLE")
arcpy.AddField_management(UNHEALTHY_FOOD_FC_PATH, "Y_Coord", "DOUBLE")
arcpy.AddField_management(UNHEALTHY_FOOD_FC_PATH,"Address","TEXT")

required_fields:list[str] = ["Address", "X_Coord", "Y_Coord"]
for required_field in required_fields:
    if any(field.name == required_field for field in arcpy.ListFields(UNHEALTHY_FOOD_FC_PATH)):
        print(f"The field '{required_field}' exists.")
    else:
        print(f"The field '{required_field}' does not exist.")

Fields in the feature UNHEALTHY_FOOD feature class:
OBJECTID
Shape
The field 'Address' exists.
The field 'X_Coord' exists.
The field 'Y_Coord' exists.


In [61]:
### Process Alt healthy food options from their JSON files ###

with open(CONVENIENCE_JSON_PATH) as file:
    CONVENIENCE_JSON = json.load(file)
with open(CORNERSTORE_JSON_PATH) as file:
    CORNERSTORE_JSON = json.load(file)
with open(MINI_MART_JSON_PATH) as file:
    MINI_MART_JSON = json.load(file)
with open(FAST_FOOD_JSON_PATH) as file:
    FAST_FOOD_JSON = json.load(file)

UNHEALTHY_FOOD_JSON = [CONVENIENCE_JSON,CORNERSTORE_JSON,MINI_MART_JSON,FAST_FOOD_JSON]

### Loop through all json files in the list ###     
for food_ops in UNHEALTHY_FOOD_JSON:
    # Step 3: Insert points into the feature class from the JSON data
    with arcpy.da.InsertCursor(UNHEALTHY_FOOD_FC_PATH, ["SHAPE", "Address", "X_Coord", "Y_Coord"]) as cursor:
        for address, coords in food_ops.items():
            # Create a point geometry object using the x and y coordinates
            point = arcpy.Point(coords[1], coords[0])  # coords[1] is longitude (X), coords[0] is latitude (Y)
            
            # Prepare the row with geometry and attributes
            row = [point, address, coords[1], coords[0]]  # X_Coord is longitude, Y_Coord is latitude
            
            # Insert the row into the feature class
            cursor.insertRow(row)

# Step 4: Create a feature layer from the feature class
UNHEALTHY_FOOD_FL = "unhealthy_food_fl"
if arcpy.Exists(UNHEALTHY_FOOD_FL):
    arcpy.Delete_management(UNHEALTHY_FOOD_FL)
    arcpy.DefineProjection_management(UNHEALTHY_FOOD_FC_PATH, SPATIAL_REF)
    arcpy.MakeFeatureLayer_management(UNHEALTHY_FOOD_FC_PATH, UNHEALTHY_FOOD_FL)
else:
    arcpy.DefineProjection_management(UNHEALTHY_FOOD_FC_PATH, SPATIAL_REF)
    arcpy.MakeFeatureLayer_management(UNHEALTHY_FOOD_FC_PATH, UNHEALTHY_FOOD_FL)